In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import models, layers
import tensorflow as tf
from ipywidgets import IntProgress
from IPython.display import display
import librosa
import tqdm
import time 
import os



def alert():
    duration = 1  # seconds
    freq = 440  # Hz

    for i in range(5):
        os.system('play -nq -t alsa synth {} sine {}'.format(duration, freq))
    os.system('spd-say "your program has finished"')
    time.sleep(2)
    for i in range(5):
        os.system('play -nq -t alsa synth {} sine {}'.format(duration, freq))
    os.system('spd-say "your program has finished"')

2024-02-29 16:19:33.443610: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-29 16:19:33.882938: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-02-29 16:19:33.882964: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-02-29 16:19:33.942830: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-29 16:19:35.121264: W tensorflow/stream_executor/platform/de

In [2]:
train_csv = pd.read_csv('./archive/Metadata_Train.csv')
test_csv = pd.read_csv('./archive/Metadata_Test.csv')
train_csv.head()

,FileName,Class
0,1-E1-Major 00.wav,Sound_Guitar
1,1-E1-Major 01.wav,Sound_Guitar
2,1-E1-Major 02.wav,Sound_Guitar
3,1-E1-Major 03.wav,Sound_Guitar
4,1-E1-Major 04.wav,Sound_Guitar


In [3]:
def get_windows(audio, window_size = 22050):
    
    start = 0
    windows = []
    audio_len = len(audio)
    
    while start < audio_len:
        
        #find window end
        if start+window_size > audio_len:
            break
        else: 
            window_end = int(start + window_size)
        #take window 
        windows.append(audio[start:window_end])
        #move window
        start += int(window_size / 2) 
      
    #stretch any windows the wrong size
    #window stretching appears not to work for now just discard any windows that are too small
    # for index, window in enumerate(windows):
    #     if  len(window) != window_size:
    #         rate = 1/(window_size / len(window))
    #         windows[index] = librosa.effects.time_stretch(y = window, rate=rate)
        
        
    return windows

training_data = {'mel spec' : [], 'instrument' : []}

for index, row in tqdm.tqdm_notebook(train_csv.iterrows(), desc = 'tqdm() Progress Bar', total = len(train_csv)):
    
    filename = row['FileName']
    audio, sr = librosa.load(path = f'./archive/Train_submission/Train_submission/{filename}')
    windowed_audio = get_windows(audio)
    
    for audio_window in windowed_audio:
        
        if len(audio_window) > 22050:
            print(index)
            while(1):
                print(".")
                pass
        
        mel = librosa.feature.melspectrogram(y=audio_window, sr=sr)
        mel_to_db = librosa.power_to_db(mel, ref=np.max)
        
        training_data['mel spec'].append(mel_to_db)
        training_data['instrument'].append(row['Class'])
        
df = pd.DataFrame.from_dict(training_data)
df.to_csv('training_data.csv')
    
os.system('spd-say "test data has compiled"')

/tmp/ipykernel_171797/2697586272.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index, row in tqdm.tqdm_notebook(train_csv.iterrows(), desc = 'tqdm() Progress Bar', total = len(train_csv)):


tqdm() Progress Bar:   0%|          | 0/2629 [00:00<?, ?it/s]

0

In [5]:
test_data = {'mel spec' : [], 'instrument' : []}

for index, row in tqdm.tqdm_notebook(test_csv.iterrows(), desc = 'tqdm() Progress Bar', total = len(test_csv)):
    
    filename = row['FileName']
    audio, sr = librosa.load(path = f'./archive/Test_submission/Test_submission/{filename}')
    windowed_audio = get_windows(audio, sr)
    
    for audio_window in windowed_audio:
        
        mel = librosa.feature.melspectrogram(y=audio_window, sr=sr)
        mel_to_db = librosa.power_to_db(mel, ref=np.max)
        
        test_data['mel spec'].append(mel_to_db)
        test_data['instrument'].append(row['Class'])
        
     
df = pd.DataFrame.from_dict(test_data)
df.to_csv('test_data.csv')

alert()
    

/tmp/ipykernel_171797/844846901.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index, row in tqdm.tqdm_notebook(test_csv.iterrows(), desc = 'tqdm() Progress Bar', total = len(test_csv)):


tqdm() Progress Bar:   0%|          | 0/80 [00:00<?, ?it/s]